**Conversion of data txt file to json file**

In [ ]:
import re
import json

# Read the contents of the text file
with open('/content/abstract-intelligen-set_copy (2).txt', 'r', encoding='utf-8') as file:
    raw_text = file.read()

def extract_paper_info(text):
    # Extracting citation information
    citation_match = re.search(r'Citation:(.*?)doi:(.*?)(?=Citation:|$)', text, re.DOTALL)
    citation = citation_match.group(1).strip() if citation_match else None

    # Extracting title information
    title_match = re.search(r'Title:(.*?)(?=Authors:|$)', text, re.DOTALL)
    title = title_match.group(1).strip() if title_match else None

    # Extracting authors information
    authors_match = re.search(r'Authors:(.*?)(?=AuthorInformation:|$)', text, re.DOTALL)
    authors = re.findall(r'(\S.*?)\s*(?:,|$)', authors_match.group(1)) if authors_match else None

    # Extracting author information
    author_info_match = re.search(r'AuthorInformation:(.*?)(?=Abstract:|$)', text, re.DOTALL)
    author_information = re.findall(r'(\d+)', author_info_match.group(1)) if author_info_match else None

    # Extracting abstract information
    abstract_match = re.search(r'Abstract:(.*?)(?=DOI:|$)', text, re.DOTALL)
    abstract = abstract_match.group(1).strip() if abstract_match else None

    # Extracting DOI information
    doi_match = re.search(r'DOI:(.*?)(?=PMCID:|$)', text, re.DOTALL)
    doi = doi_match.group(1).strip() if doi_match else None

    # Extracting PMCID information
    pmcid_match = re.search(r'PMCID:(.*?)(?=PMID:|$)', text, re.DOTALL)
    pmcid = pmcid_match.group(1).strip() if pmcid_match else None

    # Extracting PMID information
    pmid_match = re.search(r'PMID:(.*?)(?=\[Indexed for MEDLINE\]|$)', text, re.DOTALL)
    pmid = pmid_match.group(1).strip() if pmid_match else None

    return {
        "Citation": citation,
        "Title": title,
        "Authors": authors,
        "AuthorInformation": author_information,
        "Abstract": abstract,
        "DOI": doi,
        "PMCID": pmcid,
        "PMID": pmid
    }

# Split the text into individual papers
papers = re.split(r'Citation:', raw_text)[1:]

# Extract information for each paper
results = [extract_paper_info("Citation:" + paper) for paper in papers]

# Save the results in a JSON file
with open('output.json', 'w', encoding='utf-8') as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=2)

print("Results saved in 'output.json'")


Results saved in 'output.json'


**Information retrieval**

**Single query**

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the data from output.json
with open('/content/output.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Extract the abstracts from the data
abstracts = [paper['Abstract'] for paper in data]

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(abstracts)

def retrieve_documents(query, num_results=1):
    # Transform the query using the same vectorizer
    query_vector = vectorizer.transform([query])

    # Calculate cosine similarity between the query and all documents
    similarity_scores = cosine_similarity(query_vector, tfidf_matrix)[0]

    # Get the indices of the most similar documents
    most_similar_indices = similarity_scores.argsort()[-num_results:][::-1]

    # Retrieve the most similar documents
    results = [data[i] for i in most_similar_indices]

    return results

# Example query
query = "How is artificial intelligence used in medicine?"

# Retrieve documents based on the query
retrieved_documents = retrieve_documents(query, num_results=2)

# Print the retrieved documents
for i, document in enumerate(retrieved_documents, 1):
    print(f"\nResult {i}:")
    print(f"Title: {document['Title']}")
    print(f"Authors: {', '.join(document['Authors'])}")
    print(f"Abstract: {document['Abstract']}")
    print(f"DOI: {document['DOI']}")
    print(f"PMID: {document['PMID']}")



Result 1:
Title: Artificial Intelligence in Nephrology: Core Concepts, Clinical Applications, and 
Perspectives.
Authors: Niel O(1), Bastard P(2).
Abstract: Artificial intelligence is playing an increasingly important role in many fields 
of medicine, assisting physicians in most steps of patient management. In 
nephrology, artificial intelligence can already be used to improve clinical 
care, hemodialysis prescriptions, and follow-up of transplant recipients. 
However, many nephrologists are still unfamiliar with the basic principles of 
medical artificial intelligence. This review seeks to provide an overview of 
medical artificial intelligence relevant to the practicing nephrologist, in all 
fields of nephrology. We define the core concepts of artificial intelligence and 
machine learning and cover the basics of the functioning of neural networks and 
deep learning. We also discuss the most recent clinical applications of 
artificial intelligence in nephrology and medicine; as an e

**Multiple queries**

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the data from output.json
with open('/content/output.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# Extract the abstracts from the data
abstracts = [paper['Abstract'] for paper in data]

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(abstracts)

def retrieve_documents(query, num_results=1):
    # Transform the query using the same vectorizer
    query_vector = vectorizer.transform([query])

    # Calculate cosine similarity between the query and all documents
    similarity_scores = cosine_similarity(query_vector, tfidf_matrix)[0]

    # Get the indices of the most similar documents
    most_similar_indices = similarity_scores.argsort()[-num_results:][::-1]

    # Retrieve the most similar documents
    results = [data[i] for i in most_similar_indices]

    return results

# Example queries
queries = [
    "How is artificial intelligence used in medicine?",
    "ophthalmology?",
    "Emotional intelligence"
]

# Retrieve documents for each query
for i, query in enumerate(queries, 1):
    retrieved_documents = retrieve_documents(query, num_results=2)

    # Print the retrieved documents for each query
    print(f"\nResults for Query {i} - '{query}':")
    for j, document in enumerate(retrieved_documents, 1):
        print(f"\nResult {j}:")
        print(f"Title: {document['Title']}")
        print(f"Authors: {', '.join(document['Authors'])}")
        print(f"Abstract: {document['Abstract']}")
        print(f"DOI: {document['DOI']}")
        print(f"PMID: {document['PMID']}")



Results for Query 1 - 'How is artificial intelligence used in medicine?':

Result 1:
Title: Artificial Intelligence in Nephrology: Core Concepts, Clinical Applications, and 
Perspectives.
Authors: Niel O(1), Bastard P(2).
Abstract: Artificial intelligence is playing an increasingly important role in many fields 
of medicine, assisting physicians in most steps of patient management. In 
nephrology, artificial intelligence can already be used to improve clinical 
care, hemodialysis prescriptions, and follow-up of transplant recipients. 
However, many nephrologists are still unfamiliar with the basic principles of 
medical artificial intelligence. This review seeks to provide an overview of 
medical artificial intelligence relevant to the practicing nephrologist, in all 
fields of nephrology. We define the core concepts of artificial intelligence and 
machine learning and cover the basics of the functioning of neural networks and 
deep learning. We also discuss the most recent clinical a

**Forms of Questions and Answers**


In [ ]:
import json

# Read JSON data from a file
with open("/content/output.json", "r") as json_file:
    data = json.load(json_file)

# Function to answer confirmation questions (yes/no)
def answer_confirmation_question(question, data):
    # Add your logic here based on the content of the JSON data
    # Example logic: Check if specific keywords are present in the data
    if "Artificial Intelligence" in question and "Anesthesiology" in question:
        return "Yes"
    else:
        return "No"

# Function to answer factoid-type questions (what, which, when, who, how)
def answer_factoid_question(question, data):
    # Add your logic here based on the content of the JSON data
    # Example logic: Check if specific keywords are present in the data
    if "Artificial Intelligence in Anesthesiology" in question:
        return "Artificial intelligence has been advancing in fields including anesthesiology."
    # Add more factoid-type questions and answers as needed
    # ...

# Function to answer list-type questions
def answer_list_question(question, data):
    # Add your logic here based on the content of the JSON data
    # Example logic: Extract a list from the data
    if "Authors of the paper on Artificial Intelligence in Anesthesiology" in question:
        authors = data[0].get("Authors", [])
        return authors
    # Add more list-type questions and answers as needed
    # ...

# Function to answer causal questions (why or how)
def answer_causal_question(question, data):
    # Add your logic here based on the content of the JSON data
    # Example logic: Extract relevant information from the data
    if "implications of artificial intelligence for the practicing anesthesiologist" in question:
        abstract = data[0].get("Abstract", "")
        # Extract relevant information from the abstract
        return "The implications of artificial intelligence for the practicing anesthesiologist are discussed in the paper."
    # Add more causal questions and answers as needed
    # ...

# Function to answer hypothetical questions
def answer_hypothetical_question(question, data):
    # Add your logic here based on the content of the JSON data
    # Example logic: Extract relevant information from the data
    if "impact of future artificial intelligence applications in anesthesiology" in question:
        abstract = data[0].get("Abstract", "")
        # Extract relevant information from the abstract
        return "The impact of future artificial intelligence applications in anesthesiology is discussed in the paper."
    # Add more hypothetical questions and answers as needed
    # ...

# Function to answer complex questions
def answer_complex_question(question, data):
    # Add your logic here based on the content of the JSON data
    # Example logic: Extract relevant information from the data
    if "themes of applications of artificial intelligence in anesthesiology" in question:
        abstract = data[0].get("Abstract", "")
        # Extract relevant information from the abstract
        return "The scoping review identified and summarized six themes of applications of artificial intelligence in anesthesiology."
    # Add more complex questions and answers as needed
    # ...

# Example questions
confirmation_question = "Is Artificial Intelligence discussed in Anesthesiology?"
factoid_question = "What is discussed in the paper on Artificial Intelligence in Anesthesiology?"
list_question = "Who are the authors of the paper on Artificial Intelligence in Anesthesiology?"
causal_question = "What are the implications of artificial intelligence for the practicing anesthesiologist?"
hypothetical_question = "What would be the impact of future artificial intelligence applications in anesthesiology?"
complex_question = "What are the themes of applications of artificial intelligence in anesthesiology?"

# Example usage
print(answer_confirmation_question(confirmation_question, data))
print(answer_factoid_question(factoid_question, data))
print(answer_list_question(list_question, data))
print(answer_causal_question(causal_question, data))
print(answer_hypothetical_question(hypothetical_question, data))
print(answer_complex_question(complex_question, data))


Yes
Artificial intelligence has been advancing in fields including anesthesiology.
None
The implications of artificial intelligence for the practicing anesthesiologist are discussed in the paper.
The impact of future artificial intelligence applications in anesthesiology is discussed in the paper.
The scoping review identified and summarized six themes of applications of artificial intelligence in anesthesiology.


In [ ]:
import json

# Read JSON data from a file
with open("/content/output.json", "r") as json_file:
    data = json.load(json_file)

# Function to answer confirmation questions (yes/no)
def answer_confirmation_question(question, data):
    if "Artificial Intelligence" in question and "Anesthesiology" in question:
        return "Yes"
    else:
        return "No"

# Function to answer factoid-type questions (what, which, when, who, how)
def answer_factoid_question(question, data):
    if "Artificial Intelligence in Anesthesiology" in question:
        return "Artificial intelligence has been advancing in fields including anesthesiology."
    # Add more factoid-type questions and answers as needed
    # ...

# Function to answer list-type questions
def answer_list_question(question, data):
    if "Authors of the paper on Artificial Intelligence in Anesthesiology" in question:
        authors = data[0].get("Authors", [])
        return authors
    # Add more list-type questions and answers as needed
    # ...

# Function to answer causal questions (why or how)
def answer_causal_question(question, data):
    if "implications of artificial intelligence for the practicing anesthesiologist" in question:
        abstract = data[0].get("Abstract", "")
        return "The implications of artificial intelligence for the practicing anesthesiologist are discussed in the paper."
    # Add more causal questions and answers as needed
    # ...

# Function to answer hypothetical questions
def answer_hypothetical_question(question, data):
    if "impact of future artificial intelligence applications in anesthesiology" in question:
        abstract = data[0].get("Abstract", "")
        return "The impact of future artificial intelligence applications in anesthesiology is discussed in the paper."
    # Add more hypothetical questions and answers as needed
    # ...

# Function to answer complex questions
def answer_complex_question(question, data):
    if "themes of applications of artificial intelligence in anesthesiology" in question:
        abstract = data[0].get("Abstract", "")
        return "The scoping review identified and summarized six themes of applications of artificial intelligence in anesthesiology."
    # Add more complex questions and answers as needed
    # ...

# Command-line interface
while True:
    user_input = input("Ask me a question (or type 'exit' to quit): ")

    if user_input.lower() == "exit":
        break

    if "?" in user_input:
        # Identify the type of question and provide an answer
        if any(word in user_input.lower() for word in ["yes", "no"]):
            print(answer_confirmation_question(user_input, data))
        elif any(word in user_input.lower() for word in ["what", "which", "when", "who", "how"]):
            print(answer_factoid_question(user_input, data))
        elif "authors" in user_input.lower():
            print(answer_list_question(user_input, data))
        elif any(word in user_input.lower() for word in ["why", "how"]):
            print(answer_causal_question(user_input, data))
        elif "if" in user_input.lower():
            print(answer_hypothetical_question(user_input, data))
        elif any(word in user_input.lower() for word in ["and", "or"]):
            print(answer_complex_question(user_input, data))
        else:
            print("I'm sorry, I don't understand that question.")
    else:
        print("Please ask a question.")


Ask me a question (or type 'exit' to quit): what is the PMID of "Artificial Intelligence in Anesthesiology: Current Techniques, Clinical Applications, and Limitations."?
Artificial intelligence has been advancing in fields including anesthesiology.
Ask me a question (or type 'exit' to quit): Is Artificial Intelligence discussed in Anesthesiology?
None
Ask me a question (or type 'exit' to quit): exit


In [ ]:
import json

# Read JSON data from a file
with open("/content/output.json", "r") as json_file:
    data = json.load(json_file)

# Function to answer confirmation questions (yes/no)
def answer_confirmation_question(question, data):
    if "Artificial Intelligence" in data[0]["Title"] and "Anesthesiology" in data[0]["Title"]:
        return "Yes"
    else:
        return "No"

# Function to answer factoid-type questions (what, which, when, who, how)
def answer_factoid_question(question, data):
    if "Artificial Intelligence in Anesthesiology" in question:
        return data[0]["Abstract"]
    # Add more factoid-type questions and answers as needed
    # ...

# Function to answer list-type questions
def answer_list_question(question, data):
    if "Authors" in question.lower() and "Artificial Intelligence in Anesthesiology" in question:
        authors = data[0].get("Authors", [])
        return authors
    # Add more list-type questions and answers as needed
    # ...

# Function to answer causal questions (why or how)
def answer_causal_question(question, data):
    if "implications of artificial intelligence for the practicing anesthesiologist" in question:
        abstract = data[0].get("Abstract", "")
        return "The implications of artificial intelligence for the practicing anesthesiologist are discussed in the paper."
    # Add more causal questions and answers as needed
    # ...

# Function to answer hypothetical questions
def answer_hypothetical_question(question, data):
    if "impact of future artificial intelligence applications in anesthesiology" in question:
        abstract = data[0].get("Abstract", "")
        return "The impact of future artificial intelligence applications in anesthesiology is discussed in the paper."
    # Add more hypothetical questions and answers as needed
    # ...

# Function to answer complex questions
def answer_complex_question(question, data):
    if "themes of applications of artificial intelligence in anesthesiology" in question:
        abstract = data[0].get("Abstract", "")
        return "The scoping review identified and summarized six themes of applications of artificial intelligence in anesthesiology."
    # Add more complex questions and answers as needed
    # ...

# Command-line interface
while True:
    user_input = input("Ask me a question (or type 'exit' to quit): ")

    if user_input.lower() == "exit":
        break

    if "?" in user_input:
        if any(word in user_input.lower() for word in ["yes", "no"]):
            print(answer_confirmation_question(user_input, data))
        elif any(word in user_input.lower() for word in ["what", "which", "when", "who", "how"]):
            print(answer_factoid_question(user_input, data))
        elif any(word in user_input.lower() for word in ["authors"]):
            print(answer_list_question(user_input, data))
        elif any(word in user_input.lower() for word in ["why", "how"]):
            print(answer_causal_question(user_input, data))
        elif "if" in user_input.lower():
            print(answer_hypothetical_question(user_input, data))
        elif any(word in user_input.lower() for word in ["and", "or"]):
            print(answer_complex_question(user_input, data))
        else:
            print("I'm sorry, I don't understand that question.")
    else:
        print("Please ask a question.")


Ask me a question (or type 'exit' to quit): Is Artificial Intelligence discussed in Anesthesiology?
None
Ask me a question (or type 'exit' to quit): exit
